# 层和块  
层：（1）接受一组输入，（2）生成相应的输出，（3）由一组可调整参数描述。  
块：块（block）可以描述单个层、由多个层组成的组件或整个模型本身。使用块进行抽象的一个好处是可以将一些块组合成更大的组件（更大的块）。层和块的关系可以用下图描述：![layer_and_block](picture\block_and_layer.jpg)  
从编程角度，块由类（class）表示。在定义我们自己的块时，由于自动微分提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。  


### 1. 自定义块  
在实现我们自定义块之前，我们简要总结一下每个块必须提供的基本功能：  
>1.将输入数据作为其前向传播函数的参数。  
>2.通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。  
>3.计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。  
>4.存储和访问前向传播计算所需的参数。  
>5.根据需要初始化模型参数。  

在下面的代码片段中，我们从零开始编写一个块。它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。注意，下面的MLP类继承了表示块的类。我们的实现只需要提供我们自己的构造函数（Python中的__init__函数）和前向传播函数。

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义
        return self.out(F.relu(self.hidden(X)))

net = MLP()
X = torch.randn(2, 20, dtype=torch.float32, requires_grad=False)
net(X)

tensor([[-0.1933,  0.1493,  0.4787, -0.3842, -0.3123, -0.1793,  0.4839,  0.1746,
          0.0094,  0.0558],
        [-0.1403,  0.0071,  0.0176,  0.0526, -0.4227,  0.1303, -0.2228,  0.0848,
         -0.1167,  0.1527]], grad_fn=<AddmmBackward0>)

### 2. 自定义顺序块  
为了构建我们自己的简化的MySequential，我们只需要定义两个关键函数：  
> 1.一种将块逐个追加到列表中的函数；  
> 2.一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。  

In [ ]:
class MySequential(nn.Module):
    """MySequential类提供了与默认Sequential类相同的功能。"""
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 变量_modules中。 _module的类型是OrderedDict
            # 这里， module是Module子类的一个实例
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

当MySequential的前向传播函数被调用时，每个添加的块都按照它们被添加的顺序执行。现在可以使用我们的MySequential类重新实现多层感知机。  

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.4695, -0.2908,  0.3618, -0.0501, -0.2129,  0.1634,  0.1289, -0.1860,
          0.3709, -0.0069],
        [-0.1434,  0.1733,  0.4766,  0.2399,  0.0417,  0.3856,  0.0498,  0.3626,
          0.4829, -0.1937]], grad_fn=<AddmmBackward0>)

### 3. 在前向传播函数中执行自定义代码  
并不是所有的架构都是简单的顺序架构。当需要更强的灵活性时，我们需要定义自己的块。例如，我们可能希望在前向传播函数中执行Python的控制流。此外，我们可能希望执行任意的数学运算，而不是简单地依赖预定义的神经网络层。  

到目前为止，我们网络中的所有操作都对网络的激活值及网络的参数起作用。然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项，我们称之为常数参数（constant parameter）。如，我们需要一个计算函数 $f(\bold{x, w})=c\cdot \bold w^\top \bold x$的层，其中$\bold x$是输入， $\bold w$是参数， $c$是某个在优化过程中没有更新的指定常量。  

因此我们实现了一个FixedHiddenMLP类，如下所示：


In [7]:
class FixeHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand(20, 20, requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

我们可以混合搭配各种组合块的方法。在下面的例子中，我们以一些想到的方法嵌套块。

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), 
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixeHiddenMLP())
chimera(X)

tensor(-0.0448, grad_fn=<SumBackward0>)